In [ ]:
options(warn=-1)
library(dplyr)
library(tidyr)
library(data.table)
library(dtplyr)
library(lubridate)
library(ggplot2)
options(warn=0)

In [ ]:
# get pangolid data
pangolin_f <- snakemake@input$pangolin
pangolin <- fread(pangolin_f,select=c("taxon","lineage"),showProgress=TRUE,colClasses=c(taxon="character",lineage="character")) 

In [ ]:
#get meta date
meta_f <- snakemake@input$metadata
meta <- fread(meta_f,select=c("strain","date","region","country","division"),showProgress=TRUE) 


In [ ]:
# convert pangolin and meta to lazy and join them
meta <-  lazy_dt(meta)
pangolin <- lazy_dt(pangolin)
meta <- meta %>%
    left_join(pangolin, by=c("strain"="taxon")) %>%
    filter(nchar(date)==10) %>%
    as.data.table()

In [ ]:
# filter our improper date formats
meta <- meta  %>%
    filter(!grepl(pattern = "/",fixed = T, x = date)) %>%
    filter(!grepl(pattern = "X",fixed = T, x = date)) %>%
    as.data.table()
meta$date <- date(meta$date)

In [ ]:
# get floor month
meta$month = lubridate::floor_date(meta$date, "month")

In [ ]:
# get floor week
meta$week = lubridate::floor_date(meta$date, "week")

In [ ]:
fwrite(file=snakemake@output[[1]], x= meta)

In [ ]:
# for debug in case wrong data format
# for (s in meta$date){
#     tryCatch(
#         {
#             d = date(s)
#         },
#         #if an error occurs, tell me the error
#         error=function(e) {
#             message('An Error Occurred')
#             print(e)
#             print(s)
#             print("-------")
#         },
#         #if a warning occurs, tell me the warning
#         warning=function(w) {
#             message('A Warning Occurred')
#             print(w)
#             return(NA)
#         }
#     )}